# Make sure that you have 50 pickles in the [ dataset/result ] directory before running this code

In [1]:
import pandas as pd
import numpy as np
from datetime import *
import os
import pickle
import glob
import re
import time

In [2]:
# 单条路径
def sub(path_x, path_y, target, date):
    sub_df = pd.DataFrame(columns=['target','date','time','xid','yid'])
    path_length = len(path_x)
    sub_df.xid = path_x
    sub_df.yid = path_y
    sub_df.xid = sub_df.xid.astype(np.int32)
    sub_df.yid = sub_df.yid.astype(np.int32)
    sub_df.target = target
    sub_df.date = date
    # add time
    ti = datetime(2017,12,4,3,0)
    tm = [ti.strftime('%H:%M')]
    for i in range(path_length - 1):
        ti = ti + timedelta(minutes=2)
        tm.append(ti.strftime('%H:%M'))
    sub_df.time = tm
    return sub_df

In [3]:
# 多条路径合并
def sub_concat(sub_list):
    sub_csv = pd.DataFrame(columns=['target','date','time','xid','yid'])
    for i in range(len(sub_list)):
        sub_one = sub_list[i]
        sub_csv = pd.concat([sub_csv, sub_one],axis=0)
    sub_csv.target = sub_csv.target.astype(np.int32)
    sub_csv.date = sub_csv.date.astype(np.int32)
    sub_csv.xid = sub_csv.xid.astype(np.int32)
    sub_csv.yid = sub_csv.yid.astype(np.int32)
    return sub_csv

In [4]:
sub_list = []
time_cost = 60 * 24 * 50
path_count = 0
for file_path in glob.glob('../dataset/result/*.pickle'):
    filename = os.path.basename(file_path)
    if filename.startswith('part'):
        continue
    with open(file_path, 'rb') as f:
        path_list = pickle.load(f)
    path_count += 1
    time_cost = time_cost - 60 * 24 + 2 * (len(path_list) - 1)
    print ('{} is {}'.format(filename.split('.')[0], 2 * (len(path_list) - 1)))
    path_x = []
    path_y = []
    for i in range(len(path_list)):
        path_x.append(path_list[i][0])
        path_y.append(path_list[i][1])
    file_name = os.path.basename(file_path)
    date_tar = re.findall('\d+', file_name)
    sub_df = sub(path_x, path_y, int(date_tar[1]), int(date_tar[0]))
    sub_list.append(sub_df)
print ('there are {} paths. \ncost {} min'.format(path_count, time_cost))
sub_csv = sub_concat(sub_list)

day10path1 is 366
day10path10 is 602
day10path2 is 290
day10path3 is 214
day10path4 is 440
day10path5 is 730
day10path6 is 706
day10path7 is 944
day10path8 is 716
day10path9 is 182
day6path1 is 862
day6path10 is 1054
day6path2 is 594
day6path3 is 1062
day6path4 is 1050
day6path5 is 1046
day6path6 is 878
day6path7 is 880
day6path8 is 878
day6path9 is 938
day7path1 is 658
day7path10 is 1018
day7path2 is 834
day7path3 is 278
day7path4 is 404
day7path5 is 1050
day7path6 is 710
day7path7 is 1078
day7path8 is 1060
day7path9 is 778
day8path1 is 1074
day8path10 is 994
day8path2 is 1046
day8path3 is 1022
day8path4 is 1034
day8path5 is 1014
day8path6 is 1058
day8path7 is 1054
day8path8 is 1066
day8path9 is 1046
day9path1 is 412
day9path10 is 778
day9path2 is 1078
day9path3 is 238
day9path4 is 762
day9path5 is 720
day9path6 is 882
day9path7 is 1000
day9path8 is 978
day9path9 is 1018
there are 50 paths. 
cost 40574 min


In [5]:
sub_csv.to_csv('../dataset/result/result.csv', index=False, header=False)

In [6]:
df = pd.read_csv('../dataset/result/result.csv', header=None)
df.columns = ['tar_id', 'day', 'time', 'xid', 'yid']

In [7]:
def getTimeDiff(timeStra,timeStrb):
    ha, ma = timeStra.split(':')
    ha, ma = int(ha), int(ma)
    hb, mb = timeStrb.split(':')
    hb, mb = int(hb), int(mb)
    if ha > hb:
        print ('wrong order')
        return 0
    if ha == hb:
        dt = abs(ma - mb)
        if dt < 10:
            print ('**************************************************************')
            print ('danger path')
        return dt
    else:
        dt = 60 * (hb - ha - 1) + 60 - ma + mb
    return dt

In [8]:
def func(x):
    h, m = x[1].split(':')
    h, m = int(h), int(m)
    return h * 60 + m

In [9]:
del_idx = []
for day in df.day.unique():
    print ('day {}'.format(day))
    start_time = {}
    for tar in df.tar_id.unique():
        tmp = df[(df['tar_id'] == tar) & (df['day'] == day)]
        for index, row in tmp.iterrows():
            if row.xid == 142 and row.yid == 328:
                del_idx.append(index)
            else:
                start_time[tar] = df.iloc[del_idx[-1]].time
                del_idx.pop(-1)
                break
    start_time = sorted(start_time.items(), key=func)
    print (start_time)
    dt = []
    for k in range(len(start_time) - 1):
        dt.append(getTimeDiff(start_time[k][1], start_time[k + 1][1]))
    print (dt)

day 10
[(1, '03:00'), (3, '03:10'), (6, '03:20'), (8, '03:30'), (9, '03:54'), (4, '04:10'), (2, '04:30'), (5, '07:50'), (7, '08:20'), (10, '09:40')]
[10, 10, 10, 24, 16, 20, 200, 30, 80]
day 6
[(6, '03:00'), (7, '05:18'), (8, '05:28'), (2, '09:34'), (1, '11:16'), (5, '13:06'), (4, '14:28'), (9, '16:30'), (10, '17:12'), (3, '17:30')]
[138, 10, 246, 102, 110, 82, 122, 42, 18]
day 7
[(6, '03:30'), (4, '03:42'), (3, '03:58'), (8, '07:32'), (1, '07:52'), (7, '10:34'), (5, '13:10'), (2, '13:34'), (9, '13:50'), (10, '16:36')]
[12, 16, 214, 20, 162, 156, 24, 16, 166]
day 8
[(8, '07:00'), (6, '09:24'), (7, '10:10'), (5, '12:34'), (4, '14:12'), (1, '14:48'), (10, '16:12'), (3, '16:50'), (2, '17:06'), (9, '18:18')]
[144, 46, 144, 98, 36, 84, 38, 16, 72]
day 9
[(1, '03:10'), (3, '03:46'), (6, '05:40'), (7, '05:56'), (8, '07:28'), (5, '07:40'), (4, '09:40'), (10, '12:36'), (2, '17:38'), (9, '17:50')]
[36, 114, 16, 92, 12, 120, 176, 302, 12]


In [10]:
df = df.drop(del_idx)
df.to_csv('../dataset/result/result.csv', index=False, header=False)